In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_imdb_year(month, year):
    data_columns = ['imdb_title_id',
             'title',
             'original_title',
             'year',
             'date_published',
             'genre',
             'duration',
             'country',
             'language',
             'director',
             'writer',
             'production_company',
             'actors',
             'description',
             'avg_vote',
             'votes',
             'budget',
             'usa_gross_income',
             'worlwide_gross_income',
             'metascore',
             'reviews_from_users',
             'reviews_from_critics']
    
    #Initialize final DF
    df = pd.DataFrame(columns = data_columns)
    
    count = 1
    movies = 1
    website = 'https://www.imdb.com'
    
    #Stops when there are no links to find
    while movies:
        movies_url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2020-01-01,{0}-{1}-28&start={2}&ref_=adv_nxt&num_votes=100,'.format(year,month,count)
        r = requests.get(movies_url)
        r_text = r.text
        soup = BeautifulSoup(r_text, 'html.parser')
        movies = soup.find_all('div',class_='lister-item-content')
        if not movies:
            break
        count += 50
        
        imdb_title_id = ''
        title = ''
        movie_year = ''
        date_published = ''
        genre = ''
        duration = ''
        country = ''
        language = ''
        director = ''
        writer = ''
        production_company = ''
        description = ''
        avg_vote = ''
        votes = ''
        metascore = ''
        description = ''
        for movie in movies:
            movie_link = website + movie.h3.a.get('href').strip()
            imdb_title_id = movie.find('span', class_='userRatingValue')
            imdb_title_id = imdb_title_id.get('data-tconst').strip() if imdb_title_id else np.NaN
            title = movie.h3.a.get_text().strip()
            movie_year = movie.h3.find('span', class_='lister-item-year')
            movie_year = movie_year.get_text().replace('(','').replace(')','').strip() if year else np.NaN
            movie_year = re.search('\d{4}', movie_year).group()
            genre = movie.find('span', class_='genre')
            genre = genre.get_text().strip() if genre else np.NaN
            duration = movie.find('span', class_='runtime')
            duration = int(duration.get_text().replace('min','').strip()) if duration else np.NaN
            avg_vote = movie.find(itemprop='ratingValue')
            avg_vote = float(avg_vote.get('content')) if avg_vote else np.NaN
            votes = movie.find(itemprop='ratingCount')
            votes = int(votes.get('content')) if votes else np.NaN
            metascore = movie.find('span', class_='metascore')
            metascore = float(metascore.get_text().strip()) if metascore else np.NaN
            
            r1 = requests.get(movie_link)
            r1_text = r1.text
            soup1 = BeautifulSoup(r1_text, 'html.parser')
            director = soup1.find('h4', string='Director:')
            director = director.next_sibling.next_sibling.get_text().strip() if director else np.NaN
            writer = soup1.find('h4', string='Writer:')
            writer = writer.next_sibling.next_sibling.get_text().strip() if writer else np.NaN
            country = soup1.find('h4', string='Country:')
            country = country.next_sibling.next_sibling.get_text().strip() if country else np.NaN
            language = soup1.find('h4', string='Language:')
            language = language.next_sibling.next_sibling.get_text().strip() if language else np.NaN
            date_published_str = soup1.find('h4', string='Release Date:')
            if date_published_str:
                date_published_str = date_published_str.next_sibling
                date_published = re.match('(.*)\d', date_published_str)[0]
            description = soup1.find('div', class_='summary_text')
            description = description.get_text().strip() if description else np.NaN
            
            df.loc[len(df)] = [imdb_title_id, title, np.NaN, movie_year, date_published, genre, duration, country, language, director, writer,  np.NaN,  np.NaN, description, avg_vote, votes, np.NaN, np.NaN, np.NaN, metascore, np.NaN, np.NaN]
    df.to_csv('../JB Notebooks/IMDb/2020 movies onwards.csv')
    return df      


In [10]:
df = scrape_imdb_year('02','2021')